In [2]:
from pathlib import Path
cwd = str(Path.cwd().parents[0])
import sys
sys.path.append(cwd)
from engine.sql.almaden import Sql
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
import datetime
import pandas as pd
import pickle
from contextlib import suppress
import re
from html import unescape


In [4]:
# 결과 저장 경로
save_path = ""
task_id = "28320" 
# 카페 정보
cafe = {'name':"세븐나이츠"                          # 카페 이름
       ,'page_link':"https://cafe.naver.com/sevenknights"}                     # 주소
cafe.update({"keywords" : ["레볼루션"]})         # 검색 키워드

In [5]:
### 카페 주소 입력
driver = webdriver.Chrome(executable_path=f'{cwd}/chromedriver.exe')
db = Sql("datacast2")
task_id = 28320

In [19]:
driver.get(cafe.get("page_link"))

In [20]:
for keyword in cafe.get("keywords"):
    ### 카페 주소 입력

    
    ### 키워드 검색
    search_input = driver.find_element_by_css_selector("#topLayerQueryInput")
    search_input.send_keys("레볼루션")
    try :
        driver.find_element_by_xpath('//*[@id="cafe-search"]/form/button').click()   # 왼쪽에 검색창
    except:
        driver.find_element_by_xpath('//*[@id="info-search"]/form/button').click()   # 오른쪽에 검색창
    driver.implicitly_wait(0.5)
    driver.switch_to.frame('cafe_main')

    ### 키워드 수집 정보
    num_per_page = 15          # 페이지당 게시글 갯수(default: 15개)

    address_list=[]
    page = 1
    
    l=True
    while l:
        time.sleep(1)
        ### 현재 페이지의 html 불러오기
        r = driver.page_source
        page_html = BeautifulSoup(r, "html.parser")
        content = page_html.find("div", class_="article-board result-board m-tcol-c").find('tbody')
#         content = page_html.find_all("div", class_="article-board m-tcol-c")[1].find('tbody')
        body = content.find_all("tr")

        ### 게시글 정보 저장하기
        for x in body:
            temp_dict={}
            if x.find("div", class_="board-number") is not None:
                temp_dict['link'] = "https://cafe.naver.com" + x.find('a').get('href')
                temp_dict['name'] = x.find("td", class_="td_name").find('a',class_='m-tcol-c').text.strip()
                temp_dict['date'] = x.find("td", class_="td_date").text.strip().replace(".","-")[0:-1]
                temp_dict['view'] = x.find("td", class_="td_view").text.strip().replace(",","")
                if "만" in temp_dict['view']:
                    temp_dict['view'] = temp_dict['view'].replace("만","")
                    temp_dict['view'] = temp_dict['view'].replace(".","")
                    temp_dict['view'] = int(temp_dict['view'])*10000
                else : 
                    temp_dict['view'] = int(temp_dict['view'])
                print(temp_dict['view'])
                db.insert("crawl_contents",
                          task_id= task_id,
                          author = temp_dict['name'],
                          n_view = int(temp_dict['view']),
                          post_date = temp_dict['date'],
                          url=temp_dict['link'])
        print("(현재시각) "+str(datetime.datetime.now())+": "+ str(page) +"page done")

        ### 다음 페이지로 넘어가기
        page+=1
        driver.implicitly_wait(1)
        try:
            if page<=10:   # 1~10 : 페이지 번호 그대로
                page_xpath = str(page)
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            elif page == 11:   # 11 : 다음 버튼
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[11]/span').click()
            elif page>11 and page%10!=1:   # 12~ : 페이지 번호 마지막 자리 + 1
                page_xpath = str(page-((page-1)//10)*10+1)
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            elif page%10 == 1:   # 21,31.. : 다음 버튼
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[12]/span').click()
        except Exception as e:
                print(e)
                l=False


122
109
86
124
123
696
142
712
101
101
59
83
79
103
163
(현재시각) 2021-01-30 20:56:13.058163: 1page done
142
120
75
54
418
149
89
65
306
72
708
104
49
85
94
(현재시각) 2021-01-30 20:56:14.899242: 2page done
93
175
58
77
34
78
110
125
51
125
89
194
112
109
75
(현재시각) 2021-01-30 20:56:16.711377: 3page done
53
151
92
198
247
171
78
128
94
282
117
398
78
170
62
(현재시각) 2021-01-30 20:56:18.524640: 4page done
241
132
272
1122
121
79
363
131
46
156
531
156
72
146
73
(현재시각) 2021-01-30 20:56:20.328737: 5page done
2186
68
118
53
226
75
149
85
218
264
204
110
88
191
999
(현재시각) 2021-01-30 20:56:22.045105: 6page done
858
132
883
54
145
116
93
107
57
43
54
49
55
27
687
(현재시각) 2021-01-30 20:56:23.795603: 7page done
138
101
11
240
163
177
179
217
128
124
895
159
811
643
109
(현재시각) 2021-01-30 20:56:25.870465: 8page done
25
271
29
837
84
149
71
122
75
121
36
90
63
29
91
(현재시각) 2021-01-30 20:56:27.716522: 9page done
101
207
121
111
28
84
117
42
24
183
92
71
191
89
53
(현재시각) 2021-01-30 20:56:29.549627: 10page done

In [25]:
from pathlib import Path
cwd = str(Path.cwd().parents[0])
import sys
sys.path.append(cwd)
from engine.sql.almaden import Sql
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
import datetime
import pandas as pd
import pickle
from contextlib import suppress
import re
from html import unescape


# 결과 저장 경로
save_path = ""
task_id = "28320" 
# 카페 정보
cafe = {'name':"세븐나이츠"                          # 카페 이름
       ,'page_link':"https://cafe.naver.com/sevenknights"}                     # 주소
cafe.update({"keywords" : ["레볼루션"]})         # 검색 키워드

db = Sql("datacast2")
driver = webdriver.Chrome(executable_path=f'{cwd}/chromedriver.exe')
content_rows = db.select("crawl_contents","*",f"task_id={task_id}")
for keyword in cafe.get("keywords"):

    i=0
    contents_list = []   # 내용
    reply_list = []      # 댓글
    error_list = []      # 에러난 게시글

    while True:

        ### 수집 링크로 이동
        url = content_rows[i]['url']
        contents_id = content_rows[i]['contents_id']
        driver.get(url)
        time.sleep(1)
        try:
            driver.switch_to.frame('cafe_main')
            time.sleep(1)
            r = driver.page_source
            page_soup = BeautifulSoup(r, "html.parser")
            content = page_soup.find('div', class_='ArticleContentBox')  

            ### 게시글 수집
            temp_dict={}
            temp_dict['title'] = ""
            with suppress(AttributeError):   # 제목 없는 게시글
                temp_dict['title'] = content.find('h3',class_='title_text').text.strip()
            temp_dict['text'] = content.find("div", class_="article_viewer").text.strip()
            temp_dict['text'] = temp_dict['text'].replace("\n","")
            temp_dict['text'] = temp_dict['text'].replace("\u200b","")
            temp_dict['author'] = content.find('div',class_='profile_info').find('a',class_='nickname').text.strip()
            db.update_multi("crawl_contents",temp_dict,{"contents_id":contents_id})
            print(temp_dict)
            contents_list.append(temp_dict)
            
            ### 댓글 수집
            if content.find("div", class_="ReplyBox") is not None:   # 댓글 기능이 아예 없음  
                comment_num = content.find("div", class_="ReplyBox").find("a",class_="button_comment").find("strong").text
                if comment_num!='0':   # 댓글이 없음
                    comment = content.find("div", class_="CommentBox").find("ul",class_="comment_list").select("li")
                    
                    ### 댓글 구분
                    com_n=0    # 댓글
                    com_nn=0   # 대댓글
                    
                    for n in range(len(comment)):

                        if comment[n].get('class')==['CommentItem']:    # 댓글
                            com_n+=1; com_nn=0;
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn=1
                        elif comment[n].get('class')==['CommentItem', 'CommentItem--reply']:    # 대댓글
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn+=1

                        ### 댓글 내용 수집    
                        if comment[n].text.strip() != '삭제된 댓글입니다.':
                            com_nick = comment[n].find("a",class_="comment_nickname").text.strip()
                            com_date = comment[n].find("span",class_="comment_info_date").text.strip()
                            com_reply = comment[n].find("div",class_="comment_text_box").text.strip()
                            com_reply = com_reply.replace("\n","").replace("\u200b","")
                            db.insert("crawl_contents",
                                      author = com_nick,
                                      task_id = task_id,
                                      url=url,
                                      text=com_reply,
                                      is_reply=1)
            i+=1

        except:
            i+=1
            ### 게시글을 볼 등급이 안됨
            if page_soup.find('strong', class_='emph') is not None:
                error_list.append({"error" :  page_soup.find('strong', class_='emph').text+"등급 필요"
                                   , "url" : url})
                pass
            ### 에러 따로 확인
            else:
                error_list.append({"error" : "에러 확인 필요"
                                   , "url" : url})
                pass

# 크롬 종료 
driver.quit()

{'title': '세나 레볼루션은 5만명 스타트겠네', 'text': ' 지들도 잘만든 ip인거 알면서 이미지를 시원하게 말아먹노', 'author': '멜리5다스'}
{'title': '세나2 버그 방치하고 점검 길게해서 그런듯', 'text': 'ㅎㄷㄷ', 'author': '추천중'}
{'title': '레볼루션 움짤', 'text': '', 'author': '부활의 불꽃'}
{'title': '인력도 부족한 상황에서 신각 오를리 하나로 벅차지 않을까...', 'text': '그러니까 자러 갑시다', 'author': '저주의 무녀 카구라'}
{'title': '인력 빠져나간거', 'text': '첫번째 세나2 때문이고 두번째 세나가 사람이 없기때문이고세번째 세나레볼루션 때문이고 사실 사람이 없어서긴함', 'author': '룬히메'}
{'title': '섭통근데 진지하게 잘 생각하고 잘 해야댐 진짜 지금도 ㅈ대기일보직전임', 'text': '지금 섭통안하던 하던길드들중에 "길드전 못하는 길드" 생길 수 있음.아니 지금 있을걸?20명이 안되서 길드전 자체가 진행안될걸?ㅋ극단적으로 길마1명만있어도 길드전 돌아가게 해주던가 아니면 존나 기발한 아이디어로 이 상황을 타개하던가.길드전 진행을 못하는 길드 나오면 얼마나 쪽팔리겠냐?길드 해체하는거보다 더 ㅈ같을걸?님들 길드가 해체하기는 싫은데 길드원 없어서 길드전 못한다고 생각해보셈 ㅋㅋㅋㅋ그냥 그건 해체하는거보다 더 ㅈ같은거임.', 'author': '정서연'}
{'title': '세나 2? 세나 레볼루션?', 'text': '웃기지마셈 ㅋ나한테 세나는 이것밖에 없음이거 섭종하면 세창인생은 걍 끝', 'author': '프레이야님사랑해'}
{'title': '애시당초에 세나1에 인력 없다는게 나옴', 'text': '딱보자마자 진짜 세나좀했다하는사람들은 관련문제 3~5개씩은 생각해냈는데그 생각들을 개발진들이 생각하지못한 내용이 많이 포함되어있다고?다 세나2끌려감?', 'author': '모순방덱'}
{'tit

{'title': '7주년 희망사항', 'text': '1월달카일or카르마 신각2월달일영신각or구세나 출시3월달7주년 기념 세나신각4월달7주년 기념 아트북 출시5월달닼나신각6월달신규그룹 추가7월달세나의 달 기념 세나신각8월달세븐나이츠 레볼루션 출시9월달혁명단신각10월달7주년 마지막 대규모 업뎃 기념 구사황 신각11월달세븐나이츠 신작정보 공개12월달7주년 마지막 달 기념 세나신각', 'author': '부활의 불꽃'}
{'title': '방치형필드 돌리기에 내 시간과 폰 배터리가 아까움', 'text': '그래서 공기계 하나사야하나 고민중레볼루션 나오는거 보고 결정할까2개정도는 버티겠는데 게임 3개는 ㄹㅇ 못 돌림....', 'author': '닉정하기귀찮음'}
{'title': '세븐나이츠는 레볼루션 버전으로 살아날 것...', 'text': '아직안죽었어임마', 'author': '전제협회장'}
{'title': '세나2만 신경쓴다고 이젠 세나 버리는거냐?', 'text': '세나2, 레볼루션, 닌텐도까지.. 누가 뭐라해도세나 가 뿌리다. 넘 관리소홀히 하진 말아줘!', 'author': '삐사묵네'}
{'title': '세븐나이츠 레볼루션은 망할수밖에없겠네', 'text': '세븐나이츠라는 대작의 후속작이라는 카드도 세나2에서 한번정도밖에 못쓰지 그걸 2번쓰면 관심이라도 가질것같나...?세세나2도 저따구로 말아먹어놓곤..그냥 개발취소하는게 나을듯것보단 세나2 재밌네 ㅋ', 'author': '하이드리퍼 잭'}
{'title': '세나2  하느라고 세나1 전혀 못함', 'text': '레볼루션까지 나오면 진짜 감당못한다공기계 하나 만들던가 해야지', 'author': '닉정하기귀찮음'}
{'title': '세나2든 뭐든 하려면 공기계부터 사야겠음', 'text': '방치형 필드 필수겜인거 보면 레볼루션도 그런 비슷한 시스템이 있을게 뻔한데렉 안걸리고 저장공간 넉넉하게 하나쯤은 사두는게 좋겠음', 'author': '부활의 불꽃'}
{'title': '레볼루션

{'title': '세나 레볼루션은 언제 나올까', 'text': '레볼루션도 있자너', 'author': 'Joon Kim'}
{'title': '세나2 말인데', 'text': '컨셉 한참 잘못잡음1.스토리 안봐2.장르3.인게임 모델링세나 레볼루션이 원신 나오기전 출시면 인기있었듯', 'author': '룬히메'}
{'title': '세나 레볼루션땐 CM태오가 나오냐고 말이좀 많이 나오는데', 'text': '그라면 안됨 태오 이샛기 레볼루션마저 리부트 시켜버리면 답없음', 'author': '우 짱'}
{'title': '레볼루션도 인게임 다 짓밞아놓는게 아닌가 싶다', 'text': 'ㅋㅋ...', 'author': '우 짱'}
{'title': '세나2 철회하고 레볼루션이나 내라', 'text': '진짜 인게임 역대급으로 병.신이네', 'author': '딸근손실'}
{'title': '세나 레볼루션이나 기다려야것다', 'text': '', 'author': '브브 신각 기원'}
{'title': '세나2말이야 음', 'text': '퀄리티 2017 그대로면 좀 세나레볼루션이면 모를까?', 'author': '룬히메'}
{'title': '님들 제가 3년만에 왔거든여?', 'text': '물론 이 줫망비응신겜을 할라는건 아니고 오늘 세나2 사전예약이던데 할만할까싶어서 물어보러옴', 'author': '브Ro리'}
{'title': '세나2= 리니지2 레볼루션: 세나 콜라보.', 'text': '라고 쓰는게 맞지않나? 생각해봄.', 'author': 'PPiG'}
{'title': '운영자님 궁금한거 있어요', 'text': '세나 레볼루션 사전예약 언제해요?', 'author': '나만궁금한거야'}
{'title': '원신나와서 많이들접으신듯', 'text': '5200컷이 1500등ㅋㅋㅋ 44.5초가 300등ㄷㄷ길전던 시즌초인데 허허..그겜 재밋냐?', 'author': '오얏리햇반'}
{'title': '님들은 세나2나 레볼루션나오면 하실생각임?'

{'title': '넷마블 3대 철밥통중 두개가 망했구나', 'text': '모마\xa0세나\xa0린2 레볼루션\xa0\xa0이야 절대 안망할거 같던 3대장중 둘이 망했네', 'author': '현질조옷망겜'}
{'title': '세나2,세나레볼루션 4분기 출시 예정이라네.', 'text': '네이버 게임체널에 뜸.수고', 'author': '코피두잔'}
{'title': '내 생각에 대장전 이런건 세나 레볼루션에서 만들듯', 'text': '거기선 길드전 평안하게 할런지', 'author': 'Valentine'}
{'title': '토르,오딘같은 북유럽 신화 신들은 나올 수 있을까?', 'text': '딱 자라서 말하면 아닙니다. 왜냐하면 천상의 수호자 자리는 3자리가 비어는데 제4석이 라드그리드일것고 6,7석이 토르나 오딘같은 인물이 들어가긴 애매함.토르는 북유럽에서 트루드 아빠인데 딸보다 아래 북유럽에서 부모를 존경이 심각하기 때문에 천수는 아닐것같고 오딘도 절대신이라고 소개하는데 프레이야 아래? 이상함그리고 (구)천상의 수호자라는 말도 있는데 프레이야가 나오기 전에는 혼란한다고 설명해서 아닐 것 같음.\xa0\xa0마지막 엘레나처럼 설정에서만 존재하는것인데 사실상 신각해주는 이 여신이 엘레나 역활이것같음 요약해서 오딘,로키같은 신 없음 그리고 세나 레볼루션 준비중인데 스토리 빠릴 끝낼 생각같음 그럼 든!든!', 'author': '우주최강하셈무시'}
{'title': '진짜 세나=오버워치인듯', 'text': '옵치는 세나랑 비교자체가 실례인 겜이긴한데222로 인해 나락까지 떨어짐리부트로 나락까지 떨어짐필요한 패치들이 늦음옵치-신캐출시세나-여러가지 문제점 수정밸런스 못맞춤세나는 말할거도 없고 옵치도 탱힐딜 밸런스 그렇게 좋은편 아님근데 두게임 차이점이라면옵치 대표 머장님 제프카플란은 그나마 현재 블자에서 젤 정상적이라 옵치2 나올때까지 1을 어떻게든 살리려는 모습(밴픽등등)과 레딧같은데 가서 유저들 이야기듣고 체험이나 자유경쟁 만들고 그러는데여기 태가

{'title': '크섭 하소연 안해야지', 'text': '어차피 태오가 보지도않는데... 갑자기 문득 그생각이들어 삭제함\xa0그래서 세나2 출시 언제에요??', 'author': 'Joust연희'}
{'title': '그래서 세나2는 언제??', 'text': 'ㅈㄱㄴ', 'author': 'jaeyub8814'}
{'title': '뱅드림은 갓곡이 많아', 'text': '특히 파스파레볼루션', 'author': '마루야마 아야'}
{'title': '세븐나이츠 2ㆍ레볼루션 이후라면..', 'text': '다크나이츠ㆍ카일ㆍ오르카ㆍ블랙로즈ㆍ루리ㆍ에스파다ㆍ자하라가 살던 시절에서 50년 이후의 배경인데각 지역에서 사회문제들 중 하나가 주를 이루는 도시를 한 주제로 했으면 좋았을 듯ex) 부동산 폭등 지역, 전염병 지역, 교통사고 지역 ᆢ.물논 현 시점 세븐나이츠들이나 에반원정대 나이트 크로우는 동상으로 가끔씩 나오고', 'author': '엄영익명'}
{'title': '섭종글들 보니 갑자기 시무룩해짐', 'text': '나 진짜 세븐나이츠 좋아하는 최애겜인데..\xa0(물론 운영은 좀 ㅈ같지만)\xa0진짜 이대로 가다간 올해? 내년? 정도에 섭종할 것 같다는 생각이 들었음.\xa0지금도 세나말고 따로 하는 겜도 없고나름 갈아타보려고 다른 겜도 해봤는데 세나만큼 나랑 잘 맞고 재밌는 게임이 없었음.\xa0\xa0진짜 섭종하면 ㅈㄴ 슬플 것 같다. 진짜..', 'author': '빵냥이'}
{'title': '레이첼 모음', 'text': '세븐나이츠세븐나이츠2세븐나이츠 레볼루션', 'author': '부활의 불꽃'}
{'title': '세나 레볼루션 엘사 닉은 내꺼다', 'text': '뺏지마라', 'author': '엘사남친'}
{'title': '레볼루션 레이첼 모음', 'text': '신각좀', 'author': '부활의 불꽃'}
{'title': '아침을 여는 레볼루션 레이첼', 'text': '인게임 기준으로 모음', 'author': '부활의 불꽃

...

{'title': '세나2', 'text': 'MMORPG라는거 \xa0보니까 웬지 리니지 레볼루션에다가 캐릭터 덧씌워서 나오는거 아닐까 생각드네', 'author': 'echofury'}
{'title': '세나2 vip 시스템이면 바로 망할 각 ㅋㅋㅋㅋㅋㅋ', 'text': '지금까지한게 아깝지만 리니지레볼루션이랑 비슷하게 나올꺼같아서 안함 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\xa0어차피 현질을 하게끔 만들어놓을꺼같은데 ㅋ', 'author': '몬길질리네'}
{'title': '레볼루션하다 넘어왔는데 이게임 재밌나요?', 'text': '레볼루션하다 리니지 M하다 무기 날리고 재미없어져서 일로 넘어왔는데이게임 재밌나요?지금 액스인가 이게임하고 고민중이요ㅜ권력도 하루해봤는데 그럭저럭 안땡기네요..', 'author': '이겨네'}
{'title': '넷마블이 세나에 돈쓰는 이유.', 'text': '세븐나이츠는 출시후 인기있는 게임이 아니였다. 모험진행도 너무 답답하고 6성 영웅이 타 게임에 비해서 강한게 아니였기 때문 그리고 지금은 퍼주지만 초반에는 6성만들기도 너어무 힘든데 6성이 너어무 약했다 요일던전에서 주는 1성 원소들 4성 영웅 6성으로 올려서 써먹으려면 거의 한달이 걸리는 시기였음. 그래서 초반에 개망함 그냥 침몰이지..근데 세나가 살아난 이유는 49일간의 이벤트에 있었다. 솔직히 현존 하는 모든 게임의 퍼주는 이벤트는 저 49일간의 이벤트 사건이라 해도 무방할듯..발리스타가 나올때쯤 세나는 망했다 ㅇㅇ 서버 내릴 준비까지 한다는 말이 나올정도 cm루디는 그래서 섭종할 각오로 그냥 다 주고 접자 이런식으로 유저한테 다 퍼주는 이벤트를 실시 얼마나 퍼줬냐면 1년한 유저랑 신규랑 차이가 없을정도로 ㅇㅇ그결과 페이스북 트위터 등등 sns에 미친듯이 광고가 퍼지기 시작했고 답답한 진행에서 속시원한 모험이랑 결투장들이 열리면서...상상할수도 없는 신규랑 복귀 유저가 컴백하게 된다.100만명이 컴백 이사건으로 넷마블은 물론 세븐나이츠 운영진들도 너무 당황해 복귀 신규 1

{'title': '우와우', 'text': '리니지2 레볼루션 쿠폰 이벤트!리니지2 레볼루션 아데나커뮤니티에서 블루다이아 + SR 무기,방어구 쿠폰 받아가세요!리니지2 레볼루션 쿠폰 번호 - http://l2mobile.zz.am', 'author': '민박조'}
{'title': '그남친에 그여친이죠 ㅋㅋㅋ (feat 숟가R)', 'text': '코끼리가 겔리두스에 숟갈 얹힌분들이나 점수가 조금밖에 안나온 유저들한테 숟갈 얹지말라고 친히 쪽지를 보내더군요??? 하지만 이밑짤을 보면 응떠Rㅋㅋㅋㅋ 여친 안챙기고 유저인성 부터 챙기는 코끼리 인성에 브랄즈를 탁!(≥∀≤)치고갑니다. 안그래도 지금 숟가R이라고 숟가락 논란이 많은데 자기 무덤을 스스로 파고 계시네요?? ㅋㅋㅋ 정민성은 꼭보고 ㅋㅋㅋㅋㅋ+코끼리 당당함\xa0\xa01월25일 생방 팩트폭력 \xa0나:코끼리님 숟가락 얹혀도 되는거임??코끼리,시청자:응 나쁜거야 하지마 ㅎㅎ나:그럼 응떠님은 왜 숟갈얹혀요?? 아무말도 없이 칼 강퇴당함. 다음엔 부계로 팩트폭력 가할 예정임.+코리: 방송에서 공카예기 꺼내지 마세요. 여긴 제 공간입니다. 공카가 아니에요 여러분. 전 공카 신경도 안쓰고 읽지도 않아요. 욕하려면 공카에서 하세요.현실:공카오면 팩폭맞고 전치2주 쫄려서 못옴+코끼리 팩트폭력 1차(퍼옴)http://m.cafe.naver.com/ArticleRead.nhn?clubid=26903084&articleid=17521553&page=1&boardtype=L+코끼리 팩트폭력 2차(내용 별로 없음)http://m.cafe.naver.com/ArticleRead.nhn?clubid=26903084&articleid=17521673+숟가락이 뭔죄냐고 생각하는사람 들을 위해 링크검http://m.cafe.naver.com/ArticleRead.nhn?clubid=26903084&menuid=62&articleid=17521516댓글에 코리빠는 급식들 있는데 코리빨꺼면 욕만 끄적이지 말고 내글의 잘못된점을 찾아 

{'title': '레드나이츠는 재밌냐', 'text': '레볼루션은 평이 안좋던데레드나이츠는 어떤지', 'author': '허무한듸'}
{'title': '오읭? 이번주 업뎃 목요일 아닌가벼????', 'text': '리니지2 레볼루션에 올인하는겨??세나 버리는겨???', 'author': '야황육손'}
{'title': '레볼루션 씹 벌레게임 출시 후', 'text': '세나가 렉오짐 이건 서버를 빌려쓰는거냐?줫망겜 수준하곤 레볼 어차피 아재뿐이 안하는데 빨리 섭종이나하지', 'author': '카르마내놔'}
{'title': '리니지 레볼루션 미쳤네?', 'text': '뭐\xa0왠만한\xa0모발겜\xa0통틀어서\xa0핵과금유도\xa0개쩌네\xa0신규오픈게임치고\xa0야무지다\xa0야무져ㅋㅋ\xa0과금존나해라\xa0꼬우면하지말던가식이니\xa0미래가보인다ㅋㅋㅋ리니지pc처럼\xa0악세도\xa0팔기세\xa0지린다\xa0캬', 'author': '뉴월드'}
{'title': '리니지 레볼루션 진짜 실망이다', 'text': '완전 쓰레기네 역시 믿고 거르는mmorpg\xa0완전노잼 개실망', 'author': '카르마내놔'}
{'title': '리니지 레볼루션할만한데??', 'text': '세나보단 재미있어세나2년동안하니까질려죽겠다', 'author': '걸레시아'}
{'title': '레볼루션 3시간도 안남앗네..', 'text': '세나도하고~레볼도하고ㅎ\xa012시에 오픈하는겜은 \xa0첨인듯.', 'author': 'kimhy4476'}
{'title': '데차캎 팝콘 터졌어 ㅇ.ㅇ 놀러와 얘들아~', 'text': '1. 새벽에 업뎃 및 점검했는데 점검 직후에 루뽑에서 연희 오를리가 막 3~5개씩 나오는 버그가 있었음\xa0\xa02. 기존의 럼블(결투장)이 절대평가라 피로도가 존.나 높아서 랭킹1위는 게임접고 랭킹2위는 결장을 아예 안함\xa0(기존의 피로가 등급 유지조건이 500판 승률 70% 였음 랭커 또는 마스터 유지 하려먼 한판 한판 캐릭 하나

{'title': '레볼루션 길드입니다~~ 길드원 구합니다', 'text': '안녕하세요\xa0레볼루션 길드 입니다\xa0현 B상자 받고있고\xa0길드 톡방 24시간 풀 운영하고있습니다\xa0덱 및 이벤트,사냥터 서로 공유하고 있고요\xa0창설은 세나 초창기(4개월)부터 있었습니다\xa0D급 부터 같이 있던 분들로 시작하여\xa0지금은 극활성화 하고 있습니다,\xa0레이첼날은 A급 받고있습니다\xa0상위분들이 점수부족하면 루비를 써주기에\xa0하위분들이 공성에 부담없이 즐길 수 있습니다\xa0서로서로 도와가며 윈윈 하고 있고요\xa0지금점수는 최저A급상자 90%정도에 다달았습니다\xa0천천히 이끌고 가기 때문에 현재는 최소B상자로\xa0다들 만족하고 있습니다 올바른 길드 활성화죠^^\xa0같이 즐기실 길드원분 4분 모집하고있습니다\xa0점수는 8만 이상때(판당 2.5정도)\xa0공성전과 출첵은 엄격합니다\xa0규율은\xa0- 1일 미출결시 경고 및 퇴출\xa0(이유 및 급할시 말씀해주시면 다들 이해해드립니다)\xa0- 최소2주 최대3주 시간을 주어 길드원에\xa0대한 가능성판단 유.무 체크 후\xa0공성평균 점수 상향.하향 조절\xa0(발전을 기본으로 하기 때문에..9-10만 나오면\xa0해당사항 없겠죠??)우린 하향도있음ㅋㅋ\xa0- 또한 길마 혼자 길드 스킬 및 자체적 행동 자제\xa0물어보고 의견듣고 다수결\xa0관심있으신분은 카톡 dooong 톡주시길 바랍니다\xa0길드 구경도 가능!!!\xa0즐거운 세나하셔요', 'author': 'istkunst'}
{'title': '활성화 100%중인 레볼루션 길드원 뽑습니다', 'text': '안녕하세요\xa0레볼루션 길드 입니다\xa0현 B상자 받고있고\xa0길드 톡방 24시간 풀 운영하고있습니다\xa0덱 및 이벤트,사냥터 서로 공유하고 있고요\xa0창설은 세나 초창기(4개월)부터 있었습니다\xa0D급 부터 같이 있던 분들로 시작하여\xa0지금은 극활성화 하고 있습니다,\xa0레이첼날은 A급 받고있습니다\xa

IndexError: list index out of range

In [ ]:
driver.quit()